## Substituído pelo 06-rev2.py para utilização de multiprocessamento
### Modificação para evitar que dados anteriores ao intervalo de 2h sejam utilizados na amostragem
#### Necessário devido a intervalos sem resultados de análise laboratorial
#### Notebook usado para chamar funções em arquivos .py para execução em paralelo

In [23]:
# Importa as bibliotecas
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import multiprocessing
import time

In [24]:
# Importa as funções de divisão de dados
import x06a_cx_alimentacao as cx_alim
import x06b_cx_recirculacao as cx_reci
import x06c_rougher1 as rg_01
import x06d_rougher2 as rg_02
import x06e_cleaner1 as cl_01
import x06f_cleaner2 as cl_02
import x06g_recleaner as recleaner
import x06h_scavenger1b1 as sc_1b1
import x06i_scavenger1b2 as sc_1b2
import x06j_scavenger1b3 as sc_1b3
import x06k_scavenger2b1 as sc_2b1
import x06l_scavenger2b2 as sc_2b2
import x06m_scavenger2b3 as sc_2b3

In [25]:
target = pd.read_excel('T_Conceicao.xlsx')

In [4]:
%store target

Stored 'target' (DataFrame)


In [27]:
# Parâmetros que serão usados
nivel_espuma_rougher1_uso = True
vazao_ar_rougher1_uso = True
valvula_dardo1_rougher1_uso = True # False
valvula_dardo2_rougher1_uso = True # False
valvula_ar_rougher1_uso = True # False

rougher1 = pd.read_excel('Dados_Conceicao_Rougher1.xlsx')
print('Fim da leitura rougher 1')

# Transforma a primeira linha em cabeçalho
rougher1.columns = rougher1.iloc[0]

# Remove as duas primeiras linhas, que contêm cabeçalhos e nomes de TAGs
rougher1 = rougher1.drop([0,1])

# Refaz a numeração do index
rougher1.reset_index(drop=True, inplace=True)

# Preenche os dados faltantes com zeros
rougher1 = rougher1.fillna(0)

Fim da leitura rougher 1


In [28]:
base = bkp_base
fim = bkp_fim

# Seleciona duas colunas, formando um agrupamento com os dados de um parâmetro
nivel_espuma_rougher1 = rougher1.iloc[:,[0,1]]
vazao_ar_rougher1 = rougher1.iloc[:,[2,3]]
valvula_dardo1_rougher1 = rougher1.iloc[:,[4,5]]
valvula_dardo2_rougher1 = rougher1.iloc[:,[6,7]]
valvula_ar_rougher1 = rougher1.iloc[:,[8,9]]

print('RG_0001 colunas separadas')

RG_0001 colunas separadas


In [29]:
# Retira os dados nulos
selecao = (nivel_espuma_rougher1.iloc[:,0] != 0) & (nivel_espuma_rougher1.iloc[:,0] != " ") & (nivel_espuma_rougher1.iloc[:,1] != 0) & (nivel_espuma_rougher1.iloc[:,1] != "Bad")
nivel_espuma_rougher1 = nivel_espuma_rougher1[selecao]

selecao = (vazao_ar_rougher1.iloc[:,0] != 0) & (vazao_ar_rougher1.iloc[:,0] != " ") & (vazao_ar_rougher1.iloc[:,1] != 0) & (vazao_ar_rougher1.iloc[:,1] != "Bad")
vazao_ar_rougher1 = vazao_ar_rougher1[selecao]

selecao = (valvula_dardo1_rougher1.iloc[:,0] != 0) & (valvula_dardo1_rougher1.iloc[:,0] != " ") & (valvula_dardo1_rougher1.iloc[:,1] != 0) (valvula_dardo1_rougher1.iloc[:,1] != "Bad")
valvula_dardo1_rougher1 = valvula_dardo1_rougher1[selecao]

selecao = (valvula_dardo2_rougher1.iloc[:,0] != 0) & (valvula_dardo2_rougher1.iloc[:,0] != " ") & (valvula_dardo2_rougher1.iloc[:,1] != 0) & (valvula_dardo2_rougher1.iloc[:,1] != "Bad")
valvula_dardo2_rougher1 = valvula_dardo2_rougher1[selecao]

selecao = (valvula_ar_rougher1.iloc[:,0] != 0) & (valvula_ar_rougher1.iloc[:,0] != " ") & (valvula_ar_rougher1.iloc[:,1] != 0) & (valvula_ar_rougher1.iloc[:,1] != "Bad")
valvula_ar_rougher1 = valvula_ar_rougher1[selecao]

print('RG_0001 com nulos removidos')

TypeError: 'Series' object is not callable

In [33]:
valvula_dardo1_rougher1.iloc[:,1] != "Bad"

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
          ... 
504918    True
504919    True
504920    True
504921    True
504922    True
504923    True
504924    True
504925    True
504926    True
504927    True
504928    True
504929    True
504930    True
504931    True
504932    True
504933    True
504934    True
504935    True
504936    True
504937    True
504938    True
504939    True
504940    True
504941    True
504942    True
504943    True
504944    True
504945    True
504946    True
504947    True
Name: Abertura válvula dardo 1, Length: 504948, dtype: bool

In [26]:
starttime = time.time()

# Pasta para armazenamento dos dados
pasta = 'teste/'

# Data a partir da qual coletar os dados (horário da primeira análise)
# base = target.iloc[1007][0] ## Abril 01
#base = target.iloc[0][0] ## Primeiro dado
base = pd.Timestamp('2019-03-01')
#base = target.iloc[794][0]
base = base - np.timedelta64(2, 'h') ## Antes do primeiro, para que pegue todos os dados disponíveis
bkp_base = base
# Data até a qual coletar os dados (última análise)
#fim = target.iloc[1094,0] ## Abril 10
#fim = target.iloc[-1,0]
fim = pd.Timestamp('2019-03-03')
bkp_fim = fim

# Renomeia o cabeçalho de target
target.columns = ['DataHora', 'Ferro', 'Silica']

# Selecao dos target a partir da data inicial até a data final
selecao = target.iloc[:,0] > base
selecao = selecao & (target.iloc[:,0] <= fim)
target = target[selecao]

# Zera o índice
target.reset_index(drop=True, inplace=True)

target.to_excel(pasta + 'T_Conceicao.xlsx', 'Planilha1', index=False)

# Cria uma lista de processos a serem executados de forma paralela
#processes = []
#funcoes = [cx_alim, cx_reci, rg_01, rg_02, cl_01, cl_02, recleaner, sc_1b1, sc_1b2, sc_1b3, sc_2b1, sc_2b2, sc_2b3]
#funcoes = [recleaner, sc_2b1]
#funcoes = [cx_alim]

#pool = multiprocessing.Pool() #use all available cores, otherwise specify the number you want as an argument
#for func in funcoes:
#    pool.apply_async(func.executar, args=(bkp_base,bkp_fim,pasta,target,))
#pool.close()
#pool.join()

#for func in funcoes:
#    func.executar(bkp_base,bkp_fim,pasta,target)
# for func in funcoes:
    # p = multiprocessing.Process(target=func.executar, args=(bkp_base,bkp_fim,pasta,target,))
    # processes.append(p)
    # p.start()

# for process in processes:
    # process.join()
    
#print('Foram gastos {} segundos'.format(time.time() - starttime))

In [6]:
recleaner.executar(bkp_base,bkp_fim,pasta,target)

Fim da leitura recleaner


NameError: name 'cleaner2' is not defined

In [10]:
sc_2b1.executar(bkp_base,bkp_fim,pasta,target)

Fim da leitura scavenger 2 B1
SC_2B1 com nulos removidos
SC_2B1 a iniciar agrupamentos


ValueError: Lengths must match

In [ ]:
print('Foram gastos {} segundos'.format(time.time() - starttime))

x.str.strip()
isinstance(x, str)

selecao = rougher2.iloc[:,0] == isinstance(rougher2.iloc[:,0], str)
rougher2[selecao].apply(lambda x: 'no' if isinstance(x, str) else x )